In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import requests
from urllib.parse import quote
from dotenv import load_dotenv

In [16]:
api_key = os.environ.get('riot_api_key')
gameName='C9 Loki'
tagLine='kr3'
encoded_gameName = quote(gameName)
def get_puuid(encoded_gameName= None, tagLine= None, api_key =None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_gameName}/{tagLine}?api_key={api_key}'
    response = requests.get(link)
    return response.json()['puuid']

def get_ladder():
    root= 'https://na1.api.riotgames.com/'
    chall ='lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm='lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters= 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root + chall + '?api_key=' + api_key)
    gm_response = requests.get(root + gm + '?api_key=' + api_key)
    masters_response = requests.get(root + masters + '?api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries'])
    gm_df = pd.DataFrame(gm_response.json()['entries'])
    masters_df = pd.DataFrame(masters_response.json()['entries'])
    ladder = pd.concat([chall_df,gm_df,masters_df])[0:3].reset_index(drop=True)
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] += 1
    return ladder
get_ladder()


,rank,summonerId,puuid,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,VpOXU5hy0nHly6vHcEkAo7gyGzU7HT7lD7a1_QGZPill1B...,cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3Kkpqttp...,2055,244,153,True,False,False,False
1,2,QfhAYG8NzPardb0lGp-LwP-EwT4n1nAz-bTXJwWWnvH1Svg,Bb79SpPa7Kgp8v4xDIoeWAIxhfQxLEnuQxqHod_BnopMf5...,1937,218,152,True,False,False,False
2,3,SUTwQ0pFWxFUD3-U3ytXfWScuZqXbWC6ZAHcFlDuewX8UO...,NPNPxUpG8efhIvKFiniK_YTCgEnHyVMQmFh7jiqE9exdxh...,1883,144,82,True,False,False,False


In [24]:
##
import pandas as pd


df = get_ladder()



# Check the columns you care about
df = df[['summonerId', 'puuid', 'leaguePoints', 'wins', 'losses', 
         'veteran', 'inactive', 'freshBlood', 'hotStreak']]

# Save to CSV
df.to_csv('ladder_puuids.csv', index=False)
df = pd.read_csv('ladder_puuids.csv')
puuids = df['puuid'].tolist()
puuids

['cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3KkpqttpPDoL-cbd-yns_mEm6zEN1YpS-19iVc8w',
 'Bb79SpPa7Kgp8v4xDIoeWAIxhfQxLEnuQxqHod_BnopMf50ViqfC4OFKimDBeZ02O_-RhSbcncdOyw',
 'NPNPxUpG8efhIvKFiniK_YTCgEnHyVMQmFh7jiqE9exdxh7zX7y9mbrX2epFehNYfPPeoioBLWixRQ']

In [14]:
def get_match_history(region=None, puuid=None, start=0, count=50, queue=420):
    root_url = f'https://{region}.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}&queue={queue}'
    response = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key)
    return response.json()

def get_match_data_from_id(region= None,matchId=None):
    root_url= f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url + endpoint  +'?api_key=' +api_key)
    return response.json()

get_match_data_from_id(region='americas', matchId='NA1_5274110401')

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_5274110401',
  'participants': ['vEvJBCTyUBu71tTlE8yO0yO72HtQ1z9HjB78eN3ulKP6Rf4UaxLUWNE8L77noYxGs7Uaba7p8_JVzQ',
   'p9MxpDXboSK5GcEmuUg0tJsRCtto7t2yxxnTRG_WbfsTCHsbo74LQg2-SPWbRm1G0QoQU5aF_dtQtA',
   '0KGwn9FBQW3Oiz5WtkpxxMxJmMr2JVWGVsy1gTUfimOGcRr-PzTEYb-4JbHxv0Pa6PEKLz0QiFTp9g',
   'IS3bsdVPvsklO_nycE09dQYv3bb8nNY4azqnAKZyFq8R6DfmdQKLLY879ru3JnZ_wKRFa547BDs1Nw',
   'WBpudgJ8UTtsyy2J4fAd-x6OvdjEXiqztXPXHnIeX1AXJiihY8N4OHy4qvz0DG4XNX0IK8O3rmv0IQ',
   '3E1eA02idG0YrEo8JjwF0feIIomCap7Myddkt2dpczdIWnkmMDjRSXLjw0ywJKXimc2z9UBp1hx0fg',
   'JZV78JsuXEN96gjoSOG9PCA-Qp3_u9-kBhyRz39jYODCSyG2hCcI1xTAyJL-z6vdBYSEe8fCBP8W9Q',
   'cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3KkpqttpPDoL-cbd-yns_mEm6zEN1YpS-19iVc8w',
   'fMk_37v6opD1jGTmyeCNBmH5fWmHO0y1kdN_dYCboZyk76PEELzu-4Gi3VXYJ8w2o4yQLbqU2RRZ2Q',
   'ke-FataH98caWXoaZfOb2lYXtKu4BwaSJC-rPdPv8WMEeanu4igzsLNciWwceSh6afoez5Qnc-wv9w']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1745

In [26]:
def process_match_json(match_json, puuid):

    side_dict = {
        100:'blue',
        200:'red'
    }

    try:
        info = match_json['info']


        metadata = match_json['metadata']
        matchId = metadata['matchId']
        participants = metadata['participants']

        player = info['participants'][participants.index(puuid)]
        gameStartTimestamp = info['gameStartTimestamp']
        gameEndTimestamp = info['gameEndTimestamp']
        timePlayed = int((gameEndTimestamp-gameStartTimestamp)/1000//60)
        puuid = player['puuid']
  
        try:
            riotIdTagLine = player['riotIdTagline']
        except:
            riotIdTagLine = ''
        side = side_dict[player['teamId']]
        win = player['win']
        surrender = player['gameEndedInSurrender']
        totalMinionsKilled = player['totalMinionsKilled']
        totalDamageDealtToChampions = player['totalDamageDealtToChampions']
        visionScore = player['visionScore']

        matchDF = pd.DataFrame({
            'match_id': [matchId],
            'puuid': [puuid],
            'win': [win],
            'total_minions_killed': [totalMinionsKilled],
            'surrender': [surrender],
            'total_damage_dealt_champions': [totalDamageDealtToChampions],
            'vision_score': [visionScore],
            'time_played':[timePlayed],

        })
    
        return matchDF
    except:
        return pd.DataFrame()
    


In [ ]:
all_rows = []
puuidlist=['cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3KkpqttpPDoL-cbd-yns_mEm6zEN1YpS-19iVc8w']
request_count=0
for puuid in puuidlist:
    match_ids = get_match_history(region='americas',puuid=puuid,start=0, count=3) 
    request_count += 1
    time.sleep(1)

    for match_id in match_ids:
        try:
            match_json = get_match_data_from_id("americas", match_id)
            match_df = process_match_json(match_json, puuid)

            if not match_df.empty:
                all_rows.append(match_df)
            
            request_count+=1
            time.sleep(0.2)
            if request_count >= 95:
                print('Waiting for 2 min')
                time.sleep(120)
                request_count=0

        except Exception as e:
            print(f"Error with match {match_id} for puuid {puuid}: {e}")


df = pd.concat(all_rows, ignore_index=True)
print(f"Final dataframe shape: {df.shape}")
df['minionsPerMinute'] = (df['total_minions_killed'] / df['time_played']).round(2)
df['visionScorePerMinute'] = (df['vision_score'] / df['time_played']).round(2)
df['damageDealtPerMinute'] = (df['total_damage_dealt_champions'] / df['time_played']).round(2)
df #df.to_csv('challanger_data.csv', index=False)

Final dataframe shape: (3, 8)


,match_id,puuid,win,total_minions_killed,surrender,total_damage_dealt_champions,vision_score,time_played,minionsPerMinute,visionScorePerMinute,damageDealtPerMinute
0,NA1_5274110401,cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3Kkpqttp...,True,225,False,39953,33,35,6.43,0.94,1141.51
1,NA1_5274081875,cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3Kkpqttp...,True,161,True,14755,12,18,8.94,0.67,819.72
2,NA1_5273518673,cceyLZQy9agglDZRVGQX2xW9VhMjbJhY106gIr3Kkpqttp...,False,324,False,42742,37,37,8.76,1.00,1155.19
